In [1]:
using BenchmarkTools, StaticArrays, LinearAlgebra

In [2]:
A = rand(7, 7)
B = SMatrix{7, 7}(A);

In [3]:
@btime inv(A)
@btime inv(B)

  1.269 μs (4 allocations: 4.17 KiB)
  495.357 ns (1 allocation: 400 bytes)


7×7 SMatrix{7, 7, Float64, 49} with indices SOneTo(7)×SOneTo(7):
 -1.85959  -2.80359   -0.400056   0.375129    4.42414  -0.00362249   0.884385
  1.62094   2.14793   -0.514543  -0.522248   -4.35027   2.81313     -1.40824
 -1.91023  -0.960714   1.38015    0.368636    2.54625  -2.12893      1.16339
  4.84542   4.85396   -2.25796   -2.29982    -7.63666   4.16744     -1.96871
 -6.87182  -5.91524    3.19139    4.34264    12.4996   -8.11408      2.17435
 -1.89292  -2.74148    1.80179    1.17955     4.79336  -3.28688      0.824038
  6.69835   5.62348   -2.73091   -3.14729   -12.2944    6.42769     -1.28005

In [31]:
991.6/389.365

2.5467106699369486

In [4]:
function array3(fillval, N)
    fill(fillval, ntuple(d->3, N))
end

array3 (generic function with 1 method)

In [5]:
@code_warntype array3(5.0, 2)

MethodInstance for array3(::Float64, ::Int64)
  from array3(fillval, N) @ Main ~/Documents/Projects/IntroductionToJulia/test/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W4sZmlsZQ==.jl:1
Arguments
  #self#::Core.Const(array3)
  fillval::Float64
  N::Int64
Locals
  #13::var"#13#14"
Body::Any
1 ─      (#13 = %new(Main.:(var"#13#14")))
│   %2 = #13::Core.Const(var"#13#14"())
│   %3 = Main.ntuple(%2, N)::Tuple{Vararg{Int64}}
│   %4 = Main.fill(fillval, %3)::Any
└──      return %4



In [6]:
x = rand(1000, 1000);
d = sum(abs2, x; dims=2);

In [18]:
@time x ./= sqrt.(d);

  0.002613 seconds (2 allocations: 48 bytes)


In [12]:
@time let s = sqrt.(d); x ./= s end;

  0.001221 seconds (3 allocations: 7.984 KiB)


In [14]:
@time x ./= identity(sqrt.(d));

  0.001169 seconds (3 allocations: 7.984 KiB)


In [16]:
@time x ./= map(sqrt, d);

  0.001134 seconds (3 allocations: 7.984 KiB)


In [19]:
function timestep(b::Vector{T}, a::Vector{T}, Δt::T) where T
    @assert length(a)==length(b)
    n = length(b)
    b[1] = 1                            # Boundary condition
    for i=2:n-1
        b[i] = a[i] + (a[i-1] - T(2)*a[i] + a[i+1]) * Δt
    end
    b[n] = 0                            # Boundary condition
end

function heatflow(a::Vector{T}, nstep::Integer) where T
    b = similar(a)
    for t=1:div(nstep,2)                # Assume nstep is even
        timestep(b,a,T(0.1))
        timestep(a,b,T(0.1))
    end
end

heatflow(zeros(Float32,10),2)           # Force compilation
for trial=1:6
    a = zeros(Float32,1000)
    set_zero_subnormals(iseven(trial))  # Odd trials use strict IEEE arithmetic
    @time heatflow(a,1000)
end

  0.000247 seconds (1 allocation: 4.062 KiB)
  0.000114 seconds (1 allocation: 4.062 KiB)
  0.000229 seconds (1 allocation: 4.062 KiB)
  0.000115 seconds (1 allocation: 4.062 KiB)
  0.000244 seconds (1 allocation: 4.062 KiB)
  0.000123 seconds (1 allocation: 4.062 KiB)
